# 1) Get files names

* This BASH script parses the DataParsed directory and saves the names of the .txt files into `lab_files_names.txt`.

In [26]:
cd /Users/julia/bacterialGrowth_thesis/

ls -lt  DataParsed/ | grep -i .txt | awk '{print $9}' > IntermediateFiles/lab_files_names.txt

In [27]:
cd /Users/julia/bacterialGrowth_thesis/

more IntermediateFiles/lab_files_names.txt | wc -l
ls -lt DataParsed/ | grep .txt | wc -l

     107
     107


## 1.a) Modifications in files names

For the moment I will just leave the lab_files_names_modified.txt as I used it in the following scripts. But I will not modify anything, as I had incorrectly assumptions when I modified them.

In [12]:
cd /Users/julia/bacterialGrowth_thesis/IntermediateFiles
cp lab_files_names.txt lab_files_names_modified.txt

In [13]:
wc -l lab_files_names.txt
wc -l lab_files_names_modified.txt

     107 lab_files_names.txt
     107 lab_files_names_modified.txt


# 2) Get experiment names

* Extract the different experiment names fro the given directory. Several files belong to the same experiment, but we have one file per replicate.

This BASH script gets the modified names and then, by removing the last 6 characters, which are 4 for the extension and 2 for the number of replicates of one experiment, creates a file with the unique names of the experiments. These will be saved in `lab_experiment_names.txt`.
This will be useful when we try to classify the experiments depending on the metadata contained in the names the lab gave to each file. 
Also, in this classification, we will make directories with the names of the experiments that will be gotten from here.


In [14]:
awk '{print substr($1,1,length($1)-6)}' lab_files_names_modified.txt | sort | uniq | head

BTRI_WC_RNAseq
BTRI_batch_WC
BTRI_batch_mucin1
BTRI_mucin_RNAseq
BTRI_mucin_longterm
BT_BTspentWC5
BT_BTspentWC5C
BT_BTspentWC65
BT_BTspentWC65C
BT_WC_RNAseq


In [36]:
awk '{print substr($1,1,length($1)-6)}' lab_files_names_modified.txt | sort | uniq > lab_experiment_names.txt

In [37]:
wc -l lab_experiment_names.txt

      32 lab_experiment_names.txt


# 3) Get files headers

* Get the headers of the files. These fields correspond to the time, the pH measurement, bacterial abundances and metabolites

This BASH script goes through the `lab_files_names.txt` to get the true files names and look inside each of them. Each replicate has different (more or less) headers, so this script gets all of them (`lab_files_headers.txt`) and keeps the unique ones. The unique ones are contained in `lab_headers.txt`.


In [15]:
cd /Users/julia/bacterialGrowth_thesis/

rm IntermediateFiles/lab_files_headers.txt

while read -r file;
    do
        head -n 1 DataParsed/$file >> IntermediateFiles/lab_files_headers.txt
done < IntermediateFiles/lab_files_names.txt

In [16]:
cd /Users/julia/bacterialGrowth_thesis/IntermediateFiles/

head lab_files_headers.txt

more lab_files_headers.txt | wc -l

time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate liquidtotalcount liquidactivecount Neu5Ac
time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate liquidtotalcount liquidactivecount Neu5Ac
time OD pH liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate Trehalose Butyrate liquidtotalcount liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate Trehalose Butyrate liquidtotalcount liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate Trehalose Butyrate liquidtotalcount liquidactivecount liquidBTactivecount liquidRIactivecount
time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate Trehalose Butyrate liquidtotalcount liquidactivecount l

In [19]:
cat lab_files_headers.txt | tr " " "\n" | sort | uniq > lab_headers.txt

In [20]:
head lab_headers.txt

Acetate
Butyrate
Formate
Fucose
Galactose
Glucose
Lactate
Mannose
Neu5Ac
OD


# 4) Obtain experiment information

* Create an information file containing the name of each file, the experiment it belongs to and the number of replicate.
* Include more fields later!!!

Files must be name in the following way: `[experimentName]_[numberReplicate].txt` where `[experimentName]` is the name that will be placed in the database and `[numberReplicate]` the number of technical replicate for one concrete experiment.

For the moment, no information will be taken from the experiment name given in the lab in order to populate the DB.

In [34]:
rm experiments_info.txt

echo 'file_name,experiment_name,replicate_number' | column -t -s "," > experiments_info.txt

while read -r file; do

    file_name=$file
    
    experiment_name=${file_name:0:${#file_name}-6}
    
    replicate_number=${file_name:${#file_name}-5:1}
    
    echo $file_name $experiment_name $replicate_number >> experiments_info.txt
    
done < lab_files_names.txt

more experiments_info.txt | wc -l

     108


In [35]:
head experiments_info.txt

file_name  experiment_name  replicate_number
BT_batch_mucin1_1.txt BT_batch_mucin1 1
BT_batch_mucin1_4.txt BT_batch_mucin1 4
BTRI_WC_RNAseq_1.txt BTRI_WC_RNAseq 1
BTRI_WC_RNAseq_2.txt BTRI_WC_RNAseq 2
BTRI_WC_RNAseq_3.txt BTRI_WC_RNAseq 3
BTRI_batch_WC_1.txt BTRI_batch_WC 1
BTRI_batch_WC_2.txt BTRI_batch_WC 2
BTRI_batch_WC_3.txt BTRI_batch_WC 3
BTRI_batch_WC_4.txt BTRI_batch_WC 4


In [38]:
tail -n +2 experiments_info.txt > experiments_info_mod.txt

In [4]:
head experiments_info_mod.txt

BT_batch_mucin1_1.txt BT_batch_mucin1 1
BT_batch_mucin1_4.txt BT_batch_mucin1 4
BTRI_WC_RNAseq_1.txt BTRI_WC_RNAseq 1
BTRI_WC_RNAseq_2.txt BTRI_WC_RNAseq 2
BTRI_WC_RNAseq_3.txt BTRI_WC_RNAseq 3
BTRI_batch_WC_1.txt BTRI_batch_WC 1
BTRI_batch_WC_2.txt BTRI_batch_WC 2
BTRI_batch_WC_3.txt BTRI_batch_WC 3
BTRI_batch_WC_4.txt BTRI_batch_WC 4
BTRI_batch_WC_5.txt BTRI_batch_WC 5


# 5) Create directories

This BASH script creates all the directories in which the replicates’ files are going to be placed. It takes the `experiments_info_mod.txt` file and gets the columns with the experiment name and the replicate number. It first creates a directory in the following way:

`/Users/julia/bacterialGrowth_thesis/Data/$experiment/$replicate`


In [90]:
cd /Users/julia/bacterialGrowth_thesis/IntermediateFiles
while read -r line; do
    
    echo $line > line.txt

    experiment=$(awk '{print $2}' line.txt)
    replicate=$(awk '{print $3}' line.txt)

    file=$(awk '{print $1}' line.txt)

    cd /Users/julia/bacterialGrowth_thesis/Data
    mkdir -p $experiment
    mkdir -p $experiment/$replicate

    path_origin=/Users/julia/bacterialGrowth_thesis/DataParsed/"$file"
    path_destination=/Users/julia/bacterialGrowth_thesis/Data/$experiment/$replicate/
    
    cp $path_origin $path_destination
    
done < experiments_info_mod.txt

rm line.txt

# 6) Separate files into Abundances, Metabolites, pH

*Maybe it is better to pass the whole file to the Python script and in there separate it using pandas*

In [102]:
cd ../DataParsed

head BT_batch_mucin1_1.txt

time OD pH Glucose Pyruvate Succinate Lactate Formate Acetate liquidtotalcount liquidactivecount Neu5Ac
0 0.014 6.74 7.6 7.94 0.61 0.58 1.57 2.17 4753 4109 1.29
4 0.024 6.56 7.53 7.96 0.66 0.58 1.78 2.23 17257 16616 1.27
8 0.211 6.31 6.89 8.21 1.22 0.62 2.32 3.12 228160 227030 1.26
12 0.657 5.59 4.42 7.14 3.78 1.32 3.8 6.07 763902 762292 1.28
16 0.868 5.14 1.44 3.55 6.17 2.98 2.92 9.52 1728989 1726829 1.27
20 0.828 5.04 0.23 1.3 7.8 3.85 2.89 11.45 1168993 1167433 1.26
24 0.7 5.01 0.15 0.85 8.6 3.71 2.91 12.03 592834 574334 1.26
28 0.633 4.97 0.13 0.8 9.12 3.75 2.88 12.18 930189 928199 1.27
32 0.615 5 0.15 0.79 9.12 3.71 2.86 12.22 866243 863953 1.28


In [107]:
awk -v col='time' 'NR==1{for(i=1;i<=NF;i++){if($i==col){c=i;break}} print $c} NR>1{print $c}' BT_batch_mucin1_1.txt

time
0
4
8
12
16
20
24
28
32
38
42
48
72


In [108]:
awk 'FNR==1{f++}{a[f,FNR]=$1}END{for(x=1;x<=FNR;x++){for(y=1;y<ARGC;y++)printf("%s ",a[y,x]);print ""}}' BT_batch_mucin1_1.txt


time 
0 
4 
8 
12 
16 
20 
24 
28 
32 
38 
42 
48 
72 
